# Segmenting and Clustering Neighbourhoods in Toronto

## Part one:

Importing Libraries

In [6]:
import numpy as np
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

The table was scraped from the website as a .csv file which was then imported as a pandas dataframe

In [7]:
# The code was removed by Watson Studio for sharing.

In [8]:
df = pd.read_csv(body)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


I kept the cells that have an assigned Borough by removing those with 'Not assigned' in the Borough column

In [11]:
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Where more than one Neighbourhood exists in one Postcode area, the rows were then combined into one, separating the Neighbourhoods with a comma.

In [14]:
df = df.groupby('Postcode').agg({'Borough':'first', 
                             'Neighbourhood': ', '.join}).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


The cells with a borough but a 'Not assigned' neighbourhood were changed so the Neighbourhood is the same as the Borough (see 'Queens Park' in row 85).

In [15]:
df['Neighbourhood'].loc[df['Neighbourhood']== 'Not assigned'] = df['Borough']

In [17]:
df.tail(20)

,Postcode,Borough,Neighbourhood
83,M6R,West Toronto,"Parkdale, Roncesvalles"
84,M6S,West Toronto,"Runnymede, Swansea"
85,M7A,Queen's Park,Queen's Park
86,M7R,Mississauga,Canada Post Gateway Processing Centre
87,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
88,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto"
89,M8W,Etobicoke,"Alderwood, Long Branch"
90,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
91,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
92,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."


In [18]:
df.shape

(103, 3)

## Part two:

importing the geospatial_coordinates.csv into a pandas dataframe

In [34]:
# The code was removed by Watson Studio for sharing.

In [35]:
df_geo = pd.read_csv(body)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Changing the Postal Code column name on the df_geo to match the same column on df. Then merging both dataframes together as df_toronto.

In [36]:
df_geo.rename(columns={'Postal Code':'Postcode'}, inplace=True)

In [37]:
df_toronto= pd.merge(df, df_geo, on='Postcode', how='left')
df_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part three: 

Downloading all the dependencies required.

In [42]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

   

In [175]:
df_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Using geopy library to get the latitude and longitude values of Toronto

In [176]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


Creating a map of Toronto with neighbourhoods superimposed on top.

In [177]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

toronto = df_toronto

for lat, lng, borough, neighbourhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Segmenting and clustering only the neighbourhoods in Scarborough by slicing the original and creating a new dataframe for  Scarborough data.

In [178]:
scarborough_data = toronto[toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Getting the geographical coordinates of Scarborough.

In [179]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Scarborough are 43.773077, -79.257774.


Visualising Scarborough with its neighbourhoods.


In [180]:
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

Define foursquare Credentials and Version.

In [181]:
# The code was removed by Watson Studio for sharing.

Explore the first neighbourhood in Scarborough.

In [182]:
scarborough_data.loc[0,'Neighbourhood']

'Rouge, Malvern'

In [183]:
neighbourhood_latitude = scarborough_data.loc[0, 'Latitude'] 
neighbourhood_longitude = scarborough_data.loc[0, 'Longitude'] 

neighbourhood_name = scarborough_data.loc[0, 'Neighbourhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


Find the top 100 venues in Rouge, Malvern within a radius of 500 meters.

In [184]:
LIMIT = 100

radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cfd6afa351e3d128692afe8'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bb6b9446edc76b0d771311c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16e941735',
         'name': 'Fast Food Restaurant',
         'pluralName': 'Fast Food Restaurants',
         'primary': True,
         'shortName': 'Fast Food'}],
       'id': '4bb6b9446edc76b0d771311c',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside & Sheppard',
        'distance': 387,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80744841934756,
          'ln

In [185]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [186]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


In [187]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


Repeat the same process with all neighbourhoods in Scarborough and creat a new dataframe called scarborough_venues.

In [188]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [189]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                    )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


In [190]:
print (scarborough_venues.shape)
scarborough_venues.head()

(89, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [191]:
scarborough_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Clarks Corners, Sullivan, Tam O'Shanter",10,10,10,10,10,10
"Cliffcrest, Cliffside, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Scarborough Town Centre, Wexford Heights",6,6,6,6,6,6
"East Birchmount Park, Ionview, Kennedy Park",4,4,4,4,4,4


In [192]:
print('There are {} unique categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 53 unique categories.


Analyze each Neighbourhood

In [193]:
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

scarborough_onehot['Neighbourhood'] = scarborough_venues['Neighbourhood'] 

fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighbourhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Tech Startup,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [194]:
scarborough_onehot.shape

(89, 54)

In [205]:
scarborough_grouped = scarborough_onehot.groupby('Neighbourhood').mean().reset_index()
scarborough_grouped

,Neighbourhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Tech Startup,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.250000,0.0,0.0,0.00,0.000,0.250000,0.000000,0.00,0.0,0.00,0.00,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.0,0.00,0.000000,0.250,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.333333,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.0,0.00,0.000,0.000000,0.000000,0.00,0.0,0.00,0.00,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.0,0.00,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.333333,0.000000,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.000000
2,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.0,0.25,0.000,0.000000,0.000000,0.25,0.0,0.00,0.00,0.00,0.000000,0.000000,0.000,0.25,0.000000,0.000,0.000000,0.000000,0.0,0.00,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.000000,0.000,0.000000,0.000000
3,Cedarbrae,0.000000,0.000000,0.125,0.000000,0.125000,0.125,0.0,0.000000,0.0,0.0,0.00,0.125,0.000000,0.000000,0.00,0.0,0.00,0.00,0.00,0.000000,0.000000,0.125,0.00,0.000000,0.125,0.000000,0.000000,0.0,0.00,0.000000,0.125,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.125,0.000000,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.000000,0.000000,0.000,0.000000,0.200000,0.000,0.0,0.000000,0.2,0.1,0.00,0.000,0.000000,0.000000,0.00,0.0,0.00,0.00,0.00,0.000000,0.100000,0.000,0.00,0.000000,0.000,0.000000,0.100000,0.0,0.00,0.000000,0.000,0.000000,0.1,0.000000,0.000000,0.000000,0.0,0.100000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.1,0.000000,0.000000,0.000,0.000000,0.000000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.000000,0.000000,0.000,0.000000,0.000000,0.100,0.0,0.000000,0.0,0.0,0.00,0.000,0.100000,0.000000,0.00,0.0,0.00,0.00,0.00,0.000000,0.100000,0.100,0.00,0.000000,0.000,0.000000,0.000000,0.1,0.00,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.100000,0.200000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.100,0.000000,0.000000
6,"Cliffcrest, Cliffside, Scarborough Village West",0.333333,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.0,0.00,0.000,0.000000,0.000000,0.00,0.0,0.00,0.00,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.0,0.00,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.666667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.000000
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.0,0.00,0.000,0.166667,0.000000,0.00,0.0,0.00,0.00,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.333333,0.000000,0.0,0.00,0.166667,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0

In [206]:
scarborough_grouped.shape

(16, 54)

Print each neighbourhood with its 5 most common venues.

In [218]:
num_top_venues = 5

for hood in scarborough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                 venue  freq
0               Lounge  0.25
1       Breakfast Spot  0.25
2       Sandwich Place  0.25
3   Chinese Restaurant  0.25
4  American Restaurant  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
               venue  freq
0   Asian Restaurant  0.33
1               Park  0.33
2         Playground  0.33
3           Pharmacy  0.00
4  Korean Restaurant  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1  General Entertainment  0.25
2           Skating Rink  0.25
3                   Café  0.25
4            Pizza Place  0.00


----Cedarbrae----
                  venue  freq
0  Caribbean Restaurant  0.12
1    Athletics & Sports  0.12
2                Bakery  0.12
3                  Bank  0.12
4       Thai Restaurant  0.12


----Clairlea, Golden Mile, Oakridge----
           venue  freq
0         Bakery   0.2
1       Bus Line   0.2
2   Intersection   0.1
3  Metro Station   0.

Place this into a pandas dataframe in descending order, display the top 10 venues for each neighbourhood.

In [219]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [220]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = scarborough_grouped['Neighbourhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Sandwich Place,Breakfast Spot,Chinese Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Asian Restaurant,Playground,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Lounge,Hakka Restaurant,College Stadium,General Entertainment
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Intersection,Fast Food Restaurant,Soccer Field,Bus Station,Metro Station,Park,College Stadium,General Entertainment


Now running k-means to cluster the neighbourhoods into 5 clusters. Then create a new dataframe to include the cluster.

In [221]:
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 1, 4, 1, 1, 1], dtype=int32)

In [222]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

scarborough_merged = scarborough_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood',how='right')

scarborough_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3,Bar,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Intersection,Breakfast Spot,Mexican Restaurant,Tech Startup,Spa,Electronics Store,Medical Center,Pizza Place,Rental Car Location,College Stadium
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Convenience Store,Vietnamese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Lounge,Hakka Restaurant,College Stadium,General Entertainment


In [223]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine clusters and determine discriminating venue categories for each cluster - assigning a name to each cluster

#### Cluster 1: Recreation Areas

In [224]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,0,Playground,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
14,Scarborough,0,Park,Asian Restaurant,Playground,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


#### Cluster 2: Restaurants

In [225]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,1,Intersection,Breakfast Spot,Mexican Restaurant,Tech Startup,Spa,Electronics Store,Medical Center,Pizza Place,Rental Car Location,College Stadium
3,Scarborough,1,Coffee Shop,Korean Restaurant,Convenience Store,Vietnamese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,Scarborough,1,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Lounge,Hakka Restaurant,College Stadium,General Entertainment
6,Scarborough,1,Discount Store,Department Store,Playground,Coffee Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
7,Scarborough,1,Bakery,Bus Line,Intersection,Fast Food Restaurant,Soccer Field,Bus Station,Metro Station,Park,College Stadium,General Entertainment
9,Scarborough,1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
10,Scarborough,1,Indian Restaurant,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Pet Store,Bakery,Grocery Store,General Entertainment,Fried Chicken Joint,Asian Restaurant
11,Scarborough,1,Middle Eastern Restaurant,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,College Stadium,General Entertainment,Fried Chicken Joint
12,Scarborough,1,Lounge,Sandwich Place,Breakfast Spot,Chinese Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
13,Scarborough,1,Pizza Place,Noodle House,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Italian Restaurant,Bank,Chinese Restaurant,Pharmacy,Coffee Shop


#### Cluster 3: Fast Food

In [226]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store


#### Cluster 4: Bars

In [227]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3,Bar,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


#### Cluster 5: Motels

In [228]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,4,Motel,American Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
